#### Importing the libs

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')


#### Importing the relevant data sets

In [ ]:
ratings = pd.read_csv("rating_final.csv")
users = pd.read_csv("userprofile.csv")
restaurants = pd.read_csv("geoplaces2.csv")

#### See the first 5 rows of each different data set

In [ ]:
ratings.head()

In [ ]:
users.head()

In [ ]:
restaurants.head()

#### View basic information of each data set

In [ ]:
users.info()
users.describe()

In [ ]:
ratings.info()
ratings.describe()

#### Merge the data sets in order to obtain a useful one

In [ ]:
df = pd.merge(left=ratings, right=users, on="userID", how="left")
reviews_test=pd.merge(left=df, right=restaurants, on="placeID", how="left")


#### View the new reviews data set and prepare it

In [ ]:
reviews_test.head()

#### Drop useless columns

In [ ]:
reviews_test.columns

In [ ]:
reviews_test.drop([ 'state', 'country', 'fax','hijos','accessibility',
            'weight','color','franchise','ambience' ,'personality','interest','religion',
             'weight' ,'height','the_geom_meter','zip','Rambience','url'],inplace=True,axis=1)

#### Create a new column for the ratings and drop the other ones

In [ ]:
reviews_test['GlobalRatings']=reviews_test['food_rating']+reviews_test['service_rating']+reviews_test['rating']
reviews_test.drop(['food_rating','service_rating','rating'],inplace=True,axis=1)

#### Drop when the ratings is 0


In [ ]:
reviews_test.drop(reviews_test[reviews_test.GlobalRatings==0].index , inplace=True)

#### Create Age column from the Birth_year column and drop it

In [ ]:
reviews_test['age']=2019-reviews_test['birth_year']
reviews_test.drop('birth_year',inplace=True,axis=1)

#### Analyse The reviews data

In [ ]:
reviews_test.head()

In [ ]:
reviews_test.info()

In [ ]:
reviews_test.describe()

#### Replace ? with Nan

In [ ]:
reviews_test.replace('?', np.nan,inplace=True)

#### Plotting the missing values

In [ ]:
plt.figure(figsize=(8, 5))
sns.heatmap(reviews_test.isnull(),yticklabels=False,cbar=False,cmap='viridis')

#### The percentage of missing values in each column

In [ ]:
percent_missing = reviews_test.isnull().sum() * 100 / len(reviews_test)
missing_value_df = pd.DataFrame({'column_name': reviews_test.columns,
                                 'percent_missing': percent_missing})
missing_value_df['percent_missing']>50
print(missing_value_df)

#### Drop the address column since it has a 15% of empty data

In [ ]:
reviews_test.drop('address',inplace=True,axis=1)

#### Importing the geopy lib 

In [ ]:
from geopy.distance import geodesic

#### Create a small function to measure the distance between the clients and restaurants

In [ ]:
def distancer(row):
    coords_1 = (row['latitude_x'], row['longitude_x'])
    coords_2 = (row['latitude_y'], row['longitude_y'])
    return geodesic(coords_1, coords_2).km



#### Create a new comumn for the measured distance in Km

In [ ]:
reviews_test['distance'] = reviews_test.apply(distancer, axis=1)

#### View the data after modification

In [ ]:
reviews_test.head()

#### Drop the irrelevant columns

In [ ]:

reviews_test.drop(['latitude_y','longitude_y','city','latitude_x','longitude_x'],inplace=True,axis=1)

#### Create a colmun to classify the reviews

In [ ]:
reviews_test.loc[(reviews_test['GlobalRatings'] ==6) | (reviews_test['GlobalRatings'] ==5), 'Review_State'] = 'Excellent'
reviews_test.loc[(reviews_test['GlobalRatings'] ==4) | (reviews_test['GlobalRatings'] ==3), 'Review_State'] = 'Medium'
reviews_test.loc[(reviews_test['GlobalRatings'] ==2) | (reviews_test['GlobalRatings'] ==1), 'Review_State'] = 'Bad'     

#### Update the user Id as a numerical value

In [ ]:
reviews_test['userID']=reviews_test['userID'].apply(lambda x:x.split('U')[1])

In [6]:
reviews=pd.read_csv('reviewsOrg.csv')

In [7]:
reviews.head(34)

,userID,placeID,smoker,drink_level,dress_preference,transport,marital_status,activity,budget,name,alcohol,smoking_area,dress_code,price,area,other_services,GlobalRatings,age,distance,Review_State
0,1077,135085,False,social drinker,elegant,public,married,student,medium,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,medium,closed,none,6,32,0.693407,Excellent
1,1077,135038,False,social drinker,elegant,public,married,student,medium,Restaurant la Chalita,No_Alcohol_Served,section,informal,medium,closed,none,5,32,0.806876,Excellent
2,1077,132825,False,social drinker,elegant,public,married,student,medium,puesto de tacos,No_Alcohol_Served,none,informal,low,open,none,6,32,1.036330,Excellent
3,1077,135060,False,social drinker,elegant,public,married,student,medium,Restaurante Marisco Sam,No_Alcohol_Served,none,informal,medium,closed,none,5,32,0.729154,Excellent
4,1068,135104,False,casual drinker,informal,public,single,student,low,vips,Full_Bar,not permitted,informal,medium,closed,variety,4,31,0.080878,Medium
5,1068,132663,False,casual drinker,informal,public,single,student,low,tacos abi,No_Alcohol_Served,none,informal,low,closed,none,3,31,0.170468,Medium
6,1068,132630,False,casual drinker,informal,public,single,student,low,palomo tec,No_Alcohol_Served,none,informal,low,closed,none,3,31,0.427612,Medium
7,1067,132584,False,abstemious,no preference,public,single,student,medium,Gorditas Dona Tota,No_Alcohol_Served,not permitted,informal,medium,closed,none,6,32,0.338330,Excellent
8,1067,132733,False,abstemious,no preference,public,single,student,medium,Little Cesarz,No_Alcohol_Served,not permitted,informal,medium,closed,none,3,32,0.617608,Medium
9,1067,132732,False,abstemious,no preference,public,single,student,medium,Taqueria EL amigo,No_Alcohol_Served,none,casual,low,open,none,5,32,0.358171,Excellent


#### Create a single column to have the restaurant type

In [8]:
reviews['Restaurant_type']=reviews[['alcohol','smoking_area','dress_code','price','area']].apply(lambda x: '|'.join(x), axis = 1) 



#### Due to missing multiple values , replace Nan with missing 

In [9]:
reviews.replace(np.nan, 'missing',inplace=True)

#### Create a single column to have the user type

In [10]:
reviews['Client_Type']=reviews[['drink_level', 'dress_preference','transport', 'marital_status', 'activity', 'budget']].apply(lambda x: '|'.join(x), axis = 1) 

#### Exportation of the prepared data into a  new csv file

In [12]:
reviews.to_csv(r'C:\Users\hedi\Desktop\Graduation project\reviewsTest.csv', index = None, header=True)